<a href="https://colab.research.google.com/github/ThiinhUET/Fault-Detection-IoT/blob/main/FPOF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install mlxtend==0.17.2
!pip install --upgrade mlxtend
!pip show mlxtend


     |████████████████████████████████| 1.4MB 5.9MB/s 
  Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0
Name: mlxtend
Version: 0.18.0
Summary: Machine Learning Library Extensions
Home-page: https://github.com/rasbt/mlxtend
Author: Sebastian Raschka
Author-email: mail@sebastianraschka.com
License: BSD 3-Clause
Location: /usr/local/lib/python3.7/dist-packages
Requires: joblib, matplotlib, numpy, scipy, scikit-learn, pandas, setuptools
Required-by: 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import fpmax
import itertools
import copy
import numpy as np
from scipy.stats import stats
import random
import os
import sys


In [4]:
TIME_WINDOW_IN_MINUTES = 1
THRESHOLD_CV =0.9

# Identifiers starting with "LS" indicate light sensors
# Identifiers starting with "L" and "LL" indicate light switches
# Identifiers starting with "T" indicate temperature sensors
LS = ['LS013', 'LS006', 'LS010', 'LS003', 'LS004', 'LS005', 'LS015', 'LS002', 'LS014', 'LS022', 'LS016', 'LS008', 'LS007', 'LS001', 'LS020', 'LS011', 'LS017', 'LS012', 'LS021', 'LS019', 'LS018', 'LS009', 'LS023']
LL = ['LL005', 'LL001']
T = ['T104', 'T105', 'T102', 'T103', 'T101']
# Identifiers starting with "D" indicate magnetic door sensors
D = ['D002', 'D005', 'D006', 'D001', 'D003']
# Identifiers starting with "M" indicate infrared motion sensors
M = ['M021', 'M019', 'M018', 'M017', 'M012', 'M011', 'M002', 'M001', 'M022', 'M016', 'M015', 'M004', 'M006', 'M005', 'M007', 'M008']
# Identifiers starting with "MA" indicate wide-area infrared motion sensors
MA = ['MA020', 'MA013', 'MA014', 'MA010', 'MA003', 'MA009', 'MA023']

In [5]:
import pickle
from mlxtend.preprocessing import TransactionEncoder

def save_data(dataset, loc_file, pre_loc_file_url = '/content/drive/My Drive/DuLieuNhaHH102/model/'):
  loc_file = pre_loc_file_url + loc_file
  with open(loc_file, 'wb') as filehandle:
      pickle.dump(dataset, filehandle)
def load_data(loc_file, pre_loc_file_url = '/content/drive/My Drive/DuLieuNhaHH102/model/'):
  loc_file = pre_loc_file_url + loc_file
  with open(loc_file, 'rb') as filehandle:
      dataset = pickle.load(filehandle)
  return dataset


def remove_subset(list_of_list):              
    sets={frozenset(e) for e in list_of_list}  
    us=[]
    while sets:
        e=sets.pop()
        if any(e.issubset(s) for s in sets) or any(e.issubset(s) for s in us):
            continue
        else:
            us.append(sorted(list(e)))   
    return us

def convert_transaction_to_df_one_hot_vector(_transaction_dataset, sparse_format = True):
  te = TransactionEncoder()
  if(sparse_format):
    te_ary = te.fit(_transaction_dataset).transform(_transaction_dataset, sparse=True)
    return pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)
  te_ary = te.fit(_transaction_dataset).transform(_transaction_dataset)
  return pd.DataFrame(te_ary, columns=te.columns_)

def print_list_with_minK(list_of_list, min_k_item):
  count = 0
  for item in list_of_list:
    if(len(item) >= min_k_item):
      count +=1
      print(item)
  print(f"Len -origin:{len(list_of_list)} ")
  print(f"Len -filter by min_k_item={min_k_item} :{count} ")

def is_binary_device_by_df(df):
    if (df['device_id'][0:1] == 'M' 
        or df['device_id'][0:1] == 'D' 
        or df['device_id'][0:2] == 'L0'):
        val = bool(True)
    else:
        val = bool(False)
    return val
 
def is_binary_device_by_id(device_id):
    if (device_id[0:1] == 'M' 
        or device_id[0:1] == 'D' 
        or device_id[0:2] == 'L0'):
        val = bool(True)
    else:
        val = bool(False)
    return val

In [6]:
def find_groups_dev(df_transaction_dataset, min_threshold = 0.9, max_len_itemset = 10, algo = 0, _verbose = 1, _low_meomory = False):
  items  = df_transaction_dataset.columns
  related_dev_groups = []
  col_names =  ['support', 'itemsets']
  df = pd.DataFrame(columns=col_names)

  #find itemset tight with each specific item
  for item in items:
    current_group = []
    # get all transaction that contain current {item} -> make *item-transaction
    data_extract = df_transaction_dataset[df_transaction_dataset[item] == True]
    if algo == 0:
      # using fpgrowth algorithm
      frequent_itemset = fpgrowth(data_extract, min_support=min_threshold, use_colnames=True, max_len = max_len_itemset, verbose=_verbose)
    elif algo == 1:
      # using apriori algorithm
      frequent_itemset = apriori(data_extract, min_support=min_threshold, use_colnames=True ,max_len = max_len_itemset ,verbose=_verbose,low_memory = _low_meomory)
    else:
      frequent_itemset = fpmax(data_extract, min_support=min_threshold, use_colnames=True,max_len = max_len_itemset, verbose=_verbose)
    
    frames = [df, frequent_itemset]
    df = pd.concat(frames) 
    for x in frequent_itemset.itemsets:
      list_fre_items = sorted(list(x)) 
      if (item in x) & (len(x) > 1):
        current_group.append(list_fre_items)
    related_dev_groups = related_dev_groups + remove_subset(current_group)
  related_dev_groups.sort()
  related_dev_groups = list(related_dev_groups for related_dev_groups,_ in itertools.groupby(related_dev_groups))
  return related_dev_groups, df

In [5]:
def get_support(df, relate_list):
  tmp = df.copy(deep=True)
  tmp.index = pd.RangeIndex(len(tmp.index))
  for i, row in tmp.iterrows():
    if(sorted(list(row['itemsets'])) not in remove_subset(relate_list)):
      tmp.drop(i, inplace=True)
  return tmp.reset_index()
  

In [7]:
def data_preprocessing(data):

  data['device_value'] =  data['device_value'].replace(['ON', 'OPEN'], 1)
  data['device_value'] =  data['device_value'].replace(['OFF', 'CLOSE'], 0)

  data.device_value = pd.to_numeric(data.device_value, errors='raise')

  data['is_binary_device'] = data.apply(is_binary_device_by_df, axis=1)

  data.drop(data[data['device_id'].astype(str).str[0:2] == 'BA'].index, axis =0, inplace = True)

  data.drop(['ann'], axis=1, inplace =True)

  items = data['device_id'].unique()
  print(f'length of items: {len(items)}, items: {items}')
  print(data.head())
  print(data.info())

In [8]:
def get_not_NaN(row):
  if row['light_sensors']:
    return row['light_sensors']
  elif row['temp_sensors']:
    return row['temp_sensors']
  elif row['light_switch']:
    return row['light_switch']
  return NaN
  
def discrete_data(data):
  data['light_sensors'] = pd.cut(data.loc[data['device_id'].isin(LS)]['device_value'],5,labels=['1','2','3','4', '5'])
  data['temp_sensors'] = pd.cut(data.loc[data['device_id'].isin(T)]['device_value'],5,labels=['1','2','3','4', '5'])
  data['light_switch'] = pd.cut(data.loc[data['device_id'].isin(LL)]['device_value'],5,labels=['1','2','3','4', '5'])
  # for index, row in data.iterrows():
  #   data.loc[index,'mask'] = get_not_NaN(row)
  # data.assign(mask=np.where(data.light_sensors.notnull(), df.light_sensors, df.B))
  data['mask'] = np.where(data.light_sensors.notnull(), data.light_sensors, 
         (np.where(data.temp_sensors.notnull(), data.temp_sensors, data.light_switch)))
  data.drop(['light_sensors', 'temp_sensors', 'light_switch'], axis=1, inplace =True)

In [11]:
filename = '/content/drive/My Drive/DuLieuNhaHH102/ann_format.txt'
data_raw = pd.read_csv(filename, sep =' ', names = ["date", "time", "device_id", "device_value", "ann"],
                       parse_dates ={'datetime': ['date', 'time']} )

In [12]:
data = data_raw.copy(deep=True)
data_preprocessing(data)

length of items: 63, items: ['M021' 'LS013' 'MA020' 'M019' 'M018' 'MA013' 'LS006' 'LS010' 'LS003'
 'LS004' 'LS005' 'LS015' 'LS002' 'LS014' 'LS022' 'LS016' 'LS008' 'LS007'
 'LS001' 'LS020' 'LS011' 'LS017' 'LS012' 'LS021' 'LS019' 'LS018' 'M017'
 'MA014' 'M012' 'M011' 'MA010' 'MA003' 'M002' 'M001' 'D002' 'M022' 'L003'
 'M016' 'M015' 'M004' 'M006' 'M005' 'M007' 'D005' 'M008' 'T104' 'MA009'
 'L002' 'LS009' 'T105' 'MA023' 'LS023' 'D006' 'LL005' 'L005' 'L004' 'T102'
 'T103' 'T101' 'LL001' 'L001' 'D001' 'D003']
                     datetime device_id  device_value  is_binary_device
0  2011-06-15 00:06:32.834414      M021             1              True
1  2011-06-15 00:06:33.988964      M021             0              True
3  2011-06-15 00:15:01.957718     LS013             6             False
4  2011-06-15 00:25:01.892474     LS013             7             False
16 2011-06-15 03:37:46.585185      M021             1              True
<class 'pandas.core.frame.DataFrame'>
Int64Index: 407584 en

In [9]:
# save_data(data, 'ann_data.data')
data = load_data('ann_data.data')
data

,datetime,device_id,device_value,is_binary_device
0,2011-06-15 00:06:32.834414,M021,1,True
1,2011-06-15 00:06:33.988964,M021,0,True
3,2011-06-15 00:15:01.957718,LS013,6,False
4,2011-06-15 00:25:01.892474,LS013,7,False
16,2011-06-15 03:37:46.585185,M021,1,True
...,...,...,...,...
413130,2011-08-15 22:44:24.343584,LS020,23,False
413132,2011-08-15 22:54:24.303897,LS020,22,False
413133,2011-08-15 22:54:25.727398,LS021,17,False
413134,2011-08-15 23:04:25.687194,LS021,18,False


In [10]:
discrete_data(data)
data

,datetime,device_id,device_value,is_binary_device,mask
0,2011-06-15 00:06:32.834414,M021,1,True,NaN
1,2011-06-15 00:06:33.988964,M021,0,True,NaN
3,2011-06-15 00:15:01.957718,LS013,6,False,1
4,2011-06-15 00:25:01.892474,LS013,7,False,1
16,2011-06-15 03:37:46.585185,M021,1,True,NaN
...,...,...,...,...,...
413130,2011-08-15 22:44:24.343584,LS020,23,False,2
413132,2011-08-15 22:54:24.303897,LS020,22,False,2
413133,2011-08-15 22:54:25.727398,LS021,17,False,1
413134,2011-08-15 23:04:25.687194,LS021,18,False,1


In [11]:
data.loc[data['device_id'].isin(T)]

,datetime,device_id,device_value,is_binary_device,mask
1192,2011-06-15 10:00:59.781910,T104,26,False,2
1244,2011-06-15 10:10:54.873576,T104,25,False,2
1720,2011-06-15 12:19:50.801867,T104,26,False,2
2182,2011-06-15 13:05:26.390318,T105,24,False,2
2657,2011-06-15 13:39:53.809112,T105,23,False,1
...,...,...,...,...,...
408498,2011-08-15 11:29:54.689805,T105,21,False,1
408966,2011-08-15 11:56:11.629641,T104,24,False,2
409241,2011-08-15 13:09:03.643015,T105,22,False,1
412775,2011-08-15 22:11:07.567798,T104,25,False,2


In [12]:
# Gen transaction dataset
transaction_dataset = []

number_of_hours =30*24 # ~ 4 weeks
# '2011-06-15 00:06:32.834414'
start_date = data.iloc[0].datetime
# 2011-07-15 00:06:32.834414'
last_datetime =  start_date + datetime.timedelta(hours=number_of_hours)

data_to_make_group_dev = data[(data['datetime'] < last_datetime)].copy(deep=True)
start = copy.deepcopy(start_date)
num_window = 0
while start<last_datetime:
    end = start + datetime.timedelta(minutes=(TIME_WINDOW_IN_MINUTES))
    data_extract = data_to_make_group_dev[(data_to_make_group_dev['datetime'] >= start) & (data_to_make_group_dev['datetime'] < end)]
    prev_items = list()
    if not data_extract.empty:
      current_items = []
      for k,v in data_extract.iterrows():
        if not v['is_binary_device']:
          device = str(v['device_id']) + '_' + str(v['mask'])
          if not device in current_items:
            current_items.append(device)
          current_items.extend(y for y in prev_items if y not in current_items) 
        else:
          if not v['device_id'] in current_items:
            current_items.append(v['device_id'])
      prev_items = current_items  
      num_of_items = len(current_items)
      if(num_of_items>1):
        num_window +=1
        transaction_dataset.append(current_items)
      current_items = list()
        
    start = end

print(f'number of window: {num_window}')


number of window: 14109


In [13]:
transaction_dataset

[['M021', 'MA020'],
 ['MA020', 'M019', 'M018', 'MA013'],
 ['MA013', 'M018', 'MA020', 'M019', 'M021'],
 ['MA020', 'M021'],
 ['LS010_1', 'LS003_1'],
 ['LS006_1', 'LS005_1'],
 ['LS002_1', 'LS010_1', 'LS003_1'],
 ['LS016_1', 'LS008_1', 'LS006_1', 'LS005_1', 'LS007_1', 'LS001_1'],
 ['LS020_1', 'LS002_1', 'LS010_1', 'LS003_1', 'LS011_1'],
 ['LS016_1', 'LS006_1', 'LS005_1', 'LS007_1', 'LS001_1'],
 ['LS020_1', 'LS002_1', 'LS010_1', 'LS003_1', 'LS017_1', 'LS012_1', 'LS011_1'],
 ['LS006_1', 'LS005_1', 'LS007_1', 'LS001_1'],
 ['LS020_1', 'LS002_1', 'LS010_1', 'LS003_1', 'LS011_1'],
 ['LS014_1', 'LS021_1', 'M021'],
 ['LS016_1', 'LS008_1', 'LS006_1', 'LS005_1', 'LS007_1', 'LS001_1'],
 ['LS020_1', 'LS002_1', 'LS010_1', 'LS003_1', 'LS017_1', 'LS012_1', 'LS011_1'],
 ['LS014_1', 'LS021_1'],
 ['LS016_1', 'LS008_1', 'LS006_1', 'LS005_1', 'LS007_1', 'LS001_1'],
 ['LS020_1', 'LS002_1', 'LS010_1', 'LS003_1', 'LS017_1', 'LS012_1', 'LS011_1'],
 ['LS014_1', 'LS021_1'],
 ['LS016_1', 'LS008_1', 'LS006_1', 'LS005

In [ ]:
# transaction_dataset = load_data('ann_transaction_dataset_1m.data')
# group_dev = load_data('ann_group_dev_1m.data')
# print_list_with_minK(group_dev,2)
# print(group_dev)

In [14]:
df_transaction_dataset = convert_transaction_to_df_one_hot_vector(transaction_dataset)
df_transaction_dataset
# TODO: 

,D001,D002,D005,D006,L001,L002,L003,L004,L005,LL001_1,LL001_2,LL001_3,LL001_4,LL001_5,LL005_1,LL005_2,LL005_3,LL005_4,LL005_5,LS001_1,LS001_2,LS002_1,LS002_2,LS002_3,LS003_1,LS003_2,LS003_3,LS003_4,LS004_1,LS004_2,LS004_3,LS005_1,LS005_2,LS005_3,LS006_1,LS006_2,LS006_3,LS006_4,LS007_1,LS007_2,...,LS021_2,LS022_1,LS022_2,LS023_1,LS023_2,M001,M002,M004,M005,M006,M007,M008,M011,M012,M015,M016,M017,M018,M019,M021,M022,MA003,MA009,MA010,MA013,MA014,MA020,MA023,T101_5,T102_1,T102_2,T102_3,T103_1,T103_2,T103_3,T104_1,T104_2,T104_3,T105_1,T105_2
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
14105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
14106,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14107,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# group_dev = []
# group_dev,df = find_groups_dev(df_transaction_dataset,0.6, 10, algo=3, _verbose=0)

# print(f'origin: {len(group_dev)}')
# print(group_dev)
# print_list_with_minK(group_dev,2)

# group_dev = remove_subset(group_dev)
# len_group_dev =  len(group_dev)
# print(f'after remove subset:{len_group_dev} ')
# print_list_with_minK(group_dev,2)

In [ ]:
save_data(transaction_dataset, 'ann_transaction_dataset_1m.data')
save_data(group_dev, 'ann_group_dev_1m.data')

In [27]:
group_dev = []
group_dev, df = find_groups_dev(df_transaction_dataset,0.67, 10, algo=3, _verbose=0)
FPOF_input = get_support(df, group_dev)
print(FPOF_input)

    index   support                                           itemsets
0       0  0.676471                 (M001, MA003, M002, D002, LS001_1)
1       2  0.750000                           (D006, M008, D005, M007)
2      11  0.705167                       (L003, LS013_1, M018, MA013)
3      12  0.808511                       (L003, LS013_3, M018, MA013)
4      13  0.723404  (LS014_1, M017, L004, M018, MA014, LS018_1, LS...
5      16  0.777778                    (L001, MA003, LL001_2, LL001_1)
6      17  0.777778              (M001, MA003, L001, LS001_1, LL001_1)
7      18  0.777778                       (M002, L001, MA003, LL001_1)
8      19  0.777778                    (L001, MA003, LL001_2, LL001_1)
9      22  0.777778                       (M002, L001, MA003, LL001_2)
10     23  0.777778                    (LS001_1, L001, MA003, LL001_2)
11     30  0.750000           (MA003, LL001_2, M002, LS001_1, LL001_4)
12     31  0.750000              (MA003, M002, LS001_1, LL001_4, M006)
13    

In [ ]:
save_data(FPOF_input, 'FPOF_input.data')

In [ ]:
# Create test transaction
test_transaction_dataset = []

number_of_hours =2*24 # ~ 2days
# 2011-08-01 00:27:55.907856'
start_date = data.iloc[333873].datetime
# 2011-08-02 23:52:22.865609'
test_last_datetime =  start_date + datetime.timedelta(hours=number_of_hours)

test_data_to_make_group_dev = data[(data['datetime'] < test_last_datetime)].copy(deep=True)
test_start = copy.deepcopy(start_date)
test_num_window = 0
while test_start < test_last_datetime:
    test_end = test_start + datetime.timedelta(minutes=(TIME_WINDOW_IN_MINUTES))
    test_data_extract = test_data_to_make_group_dev[(test_data_to_make_group_dev['datetime'] >= test_start) & (test_data_to_make_group_dev['datetime'] < test_end)]
    if not test_data_extract.empty:
      test_current_items = list(test_data_extract.device_id.unique())
      num_of_items = len(test_current_items)
      if(num_of_items>1):
        test_num_window +=1
        test_transaction_dataset.append(test_current_items)
        
    test_start = test_end

print(f'number of window: {test_num_window}')

In [ ]:
# save_data(test_transaction_dataset, 'test_transaction_dataset.data')
df_test_transaction_dataset = convert_transaction_to_df_one_hot_vector(test_transaction_dataset)
df_test_transaction_dataset

In [ ]:
# FPOF
#' @param data \code{data.frame} or \code{transactions} from \code{arules} with input data
#' @param minSupport minimum support for FPM
def preFPOF(transaction_db):
  transaction_db.columns[      
    (transaction_db == 1)        # mask 
    .any(axis=0)     # mask
  ]
  df_dict = dict(
    list(
        transaction_db.groupby(transaction_db.index)
    )
  )
  list_items = []
  for k, v in df_dict.items():               # k: name of index, v: is a df
    check = v.columns[(v == 1).any()]
    if len(check) > 0:
        list_items.append(check.to_list())
  df = pd.DataFrame({'items':list_items})      
  # print(type(list_items))
  return df   
  


In [ ]:
def FPOF(fps, transaction_db):
  my_df = preFPOF(df_test_transaction_dataset)
  fps_lenght = len(fps)
  tmp = my_df.copy(deep=True)
  for k, v in tmp.iterrows():
    score = 0
    for i, row in fps.iterrows():
      if(set(row['itemsets']).issubset(set(v['items']))):
        score = score + row['support']
    tmp.loc[k,'FPOF_value'] = score / fps_lenght
  return tmp


In [ ]:
tmp = FPOF(FPOF_input, df_test_transaction_dataset)
print(tmp)

In [ ]:
final_df = tmp.sort_values(by=['FPOF_value'], ascending=False)
print(final_df.loc[final_df['FPOF_value'] > 0])

- cắt range của các cảm biến
- tìm thời điểm mà tất cả các cảm biến có giá trị
- Thay đổi cấu trúc của database test gắn thêm label
- tìm frequent pattern của các cảm biến ứng với từng giá trị
- chỉ xét 1 lỗi tại 1 time windown
- xác định lỗi theo rank FPOF value > 0
- thử min_support của FPmax. (0.6-0.75) 
